# Melanoma Segmentation 

In [1]:
import cv2
import os
import pickle

import numpy as np
import random
from keras.layers import Input, concatenate, Conv2DTranspose
from keras import backend as K
from keras.optimizers import Adam
import keras
from sklearn.cross_validation import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
smooth = 1

if not os.path.exists('pickled'):
    os.makedirs('pickled')

Using TensorFlow backend.


In [2]:
#Data Loading
data = []
labels = []
if not os.path.exists('pickled/seg_data.pkl'):
    for file in os.listdir('melanoma'):        
        image = cv2.imread('melanoma/' + file)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt/' + file.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)

    for file in os.listdir("others"):
        image = cv2.imread('others/' + file)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt/' + file.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)
        
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickled/seg_data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickled/seg_data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']

In [3]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels, dtype="float") / 255.0

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.30, random_state=42)

# DEFINING METRICS


In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [5]:
batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1600, 64, 64, 3)
1600 train samples
400 test samples


In [6]:
def down(filters, inputs):
    down_ = Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    down_res = Conv2D(32, (3, 3), activation='relu', padding='same')(down_)
    down_pool = MaxPooling2D(pool_size=(2, 2))(down_)
    return down_res, down_pool
    
def up(filters, inputs, down_):
    up_ = concatenate([Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(inputs), down_], axis=3)
    up_ = Conv2D(256, (3, 3), activation='relu', padding='same')(up_)
    up_ = Conv2D(256, (3, 3), activation='relu', padding='same')(up_)
    return up_

batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

inputs = Input((img_rows, img_cols, 3))

down0a, down0a_res = down(2, inputs)
down0, down0_res = down(4, down0a)
down1, down1_res = down(8, down0)
down2, down2_res = down(16, down1)
down3, down3_res = down(32, down2)


center = Conv2D(128, (3, 3), padding='same')(down3)
center = BatchNormalization(epsilon=1e-4)(center)
center = Activation('relu')(center)

up3 = up(32, center, down3_res)
up2 = up(16, up3, down2_res)
up1 = up(8, up2, down1_res)
up0 = up(4, up1, down0_res)
up0a = up(2, up0, down0a_res)

classify = Conv2D(num_classes, (1, 1), activation='sigmoid', name='final_layer')(up0a)

model = Model(inputs=inputs, outputs=classify)

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

In [7]:
model = Model(inputs=[inputs], outputs=[classify])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Score:', score)

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 435s 272ms/step - loss: -0.2746 - dice_coef: 0.2746 - val_loss: -0.2958 - val_dice_coef: 0.2958
Epoch 2/10
1600/1600 [==============================] - 355s 222ms/step - loss: -0.2758 - dice_coef: 0.2758 - val_loss: -0.2971 - val_dice_coef: 0.2971
Epoch 3/10
1600/1600 [==============================] - 499s 312ms/step - loss: -0.2771 - dice_coef: 0.2771 - val_loss: -0.2989 - val_dice_coef: 0.2989
Epoch 4/10
1600/1600 [==============================] - 356s 222ms/step - loss: -0.2796 - dice_coef: 0.2796 - val_loss: -0.3044 - val_dice_coef: 0.3044
Epoch 5/10
1600/1600 [==============================] - 359s 224ms/step - loss: -0.3076 - dice_coef: 0.3076 - val_loss: -0.4125 - val_dice_coef: 0.4125
Epoch 6/10
1600/1600 [==============================] - 365s 228ms/step - loss: -0.5028 - dice_coef: 0.5028 - val_loss: -0.6003 - val_dice_coef: 0.6003
Epoch 7/10
1600/1600 [===================